In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output


file_path = 'Desktop/IAQ_Master_file/Master_File_TVOC.xlsx'  
df = pd.read_excel(file_path, sheet_name='tvoc_2')


df = df.dropna(subset=['category', 'sub_category', 'tvoc', 'date_time'])


df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
df = df.dropna(subset=['date_time'])

df['tvoc'] = pd.to_numeric(df['tvoc'], errors='coerce')


df['hour'] = df['date_time'].dt.hour


hourly_avg = df.groupby(['category', 'sub_category', 'hour'])['tvoc'].mean().reset_index()

# debugging: Print the first few rows of hourly_avg
print("Hourly Average DataFrame:")
print(hourly_avg.head())

# Dash application
app = Dash(__name__)

# layout of the app
app.layout = html.Div([
    html.H1('TVOC Concentration Dashboard', style={'text-align': 'center'}),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': cat, 'value': cat} for cat in hourly_avg['category'].unique()],
        value=hourly_avg['category'].unique()[0],
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(id='tvoc-graph')
])

# Define callback to update graph
@app.callback(
    Output('tvoc-graph', 'figure'),
    [Input('category-dropdown', 'value')]
)
def update_graph(selected_category):
    filtered_df = hourly_avg[hourly_avg['category'] == selected_category]
    
    # Debugging: Print the filtered dataframe
    print(f"Filtered DataFrame for category {selected_category}:")
    print(filtered_df.head())
    
    fig = px.line(filtered_df, x='hour', y='tvoc', color='sub_category', markers=True,
                  title=f'Hourly Average TVOC Concentration for {selected_category}',
                  labels={'hour': 'Hour of the Day', 'tvoc': 'Average TVOC (ppm)'})
    fig.update_layout(
        template='plotly_white',
        xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Display all hours from 0 to 23
        yaxis=dict(rangemode='tozero'),  # Ensure y-axis starts from zero
        margin=dict(l=40, r=40, t=40, b=40),
        title_x=0.5
    )
    return fig


if __name__ == '__main__': # Run the app
    app.run_server(debug=True)

C:\Users\Atique\AppData\Local\Temp\ipykernel_10660\3815688811.py:13: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')


Hourly Average DataFrame:
     category  sub_category  hour      tvoc
0  cinemahall  singlescreen     0  0.018421
1  cinemahall  singlescreen     1  0.018643
2  cinemahall  singlescreen     2  0.018223
3  cinemahall  singlescreen     3  0.018317
4  cinemahall  singlescreen     4  0.019453
